In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,TimeDistributed,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

This code block is used to mount your Google Drive to the Colab environment. It allows you to access files stored in your Google Drive directly from the Colab notebook. The force_remount=True parameter ensures that the drive is remounted if it was previously mounted during the session. This is useful for ensuring that the latest version of the drive's contents is accessible.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


This code block imports essential libraries and modules for data processing, machine learning model building, and text analysis. It includes libraries for data manipulation (Pandas), neural network construction (Keras, TensorFlow), natural language processing (NLTK), and various utilities for handling arrays, strings, and file paths. Additionally, it sets the seed for random number generation to ensure reproducibility.

In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/guardian_articles.csv']

In [ ]:
pip install transformers


In [ ]:
data = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# Basic preprocessing
data['bodyContent'] = data['bodyContent'].apply(lambda x: '[CLS] ' + x + ' [SEP]')
data['webTitle'] = data['webTitle'].apply(lambda x: '[CLS] ' + x + ' [SEP]')

# Splitting the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(data['bodyContent'], data['webTitle'], test_size=0.2)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))


In [ ]:
# Load pre-trained BERT
bert = TFBertModel.from_pretrained('bert-base-uncased')

# Build a custom model
input_ids = tf.keras.Input(shape=(512,), dtype='int32', name='input_ids')  # Adjust the shape as needed
attention_mask = tf.keras.Input(shape=(512,), dtype='int32', name='attention_mask')  # Adjust the shape as needed

# BERT output
bert_output = bert(input_ids, attention_mask=attention_mask)[0]  # Using the last hidden state

# BiLSTM Decoder
decoder_bilstm = Bidirectional(LSTM(256, return_sequences=True, return_state=True))
decoder_outputs, forward_h, forward_c, backward_h, backward_c = decoder_bilstm(bert_output)

# Dense layer for output
decoder_dense = Dense(num_classes, activation='softmax')  # num_classes is the size of your target vocabulary
decoder_outputs = TimeDistributed(decoder_dense)(decoder_outputs)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)